# Import libraries

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
import numpy as np

# Database creations using pytorch Dataset 

In [8]:
class ImageAuthenticityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(os.getcwd(), self.data.iloc[idx, 3])  # image_path column
        image = Image.open(img_name).convert('RGB')
        authenticity = self.data.iloc[idx, 1]  # Authenticity column
        labels = torch.tensor([authenticity], dtype=torch.float)


        if self.transform:
            image = self.transform(image)

        return image, labels


# Definitions of the models

In [9]:
class AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)  # Predict authenticity
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations for the ImageNet dataset
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

annotations_file = 'Dataset/AIGCIQA2023/real_images_annotations.csv'

# Create the dataset
dataset = ImageAuthenticityDataset(csv_file=annotations_file, transform=data_transforms)

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


# Create data loaders
BATCH_SIZE = 1 # Set to 1 for handling individual images

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# create a susset of the test dataset for testing
test_subset_size = 10
test_subset_indices = np.random.choice(len(test_dataset), test_subset_size, replace=False)
test_subset = torch.utils.data.Subset(test_dataset, test_subset_indices)
test_subset = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Create a dictionary containing the data loaders
dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader,
    'test': test_dataloader
}

model = AuthenticityPredictor()
criterion = nn.MSELoss()  # Mean Squared Error Loss (regression)
optimizer = optim.Adam(model.regression_head.parameters(), lr=0.001)



## Compute for each image the importace scores of lastConv layer's channels

In [11]:
def compute_obj_x_obj_feature_map_importance(model, dataloader, device, layer_name):
    """Computes the importance of each feature map in a convolution layer by measuring the change in 
    predictions when the feature map is zeroed out, calculated per object.
    
    Args:
        model (nn.Module): The trained model
        dataloader (DataLoader): Dataloader containing the images (with batch_size=1)
        device (str): Device to run computation on ('cuda' or 'cpu')
        layer_name (str): Name of the layer to analyze
        
    Returns:
        numpy.ndarray: An array where each element is a numpy array of channel importance scores
                      for the corresponding image. Shape: [num_images]
    """
    # Check if importance scores are already computed
    if os.path.exists('Ranking_arrays/obj_x_obj_authenticity_importance_scores.npy'):
        print("Per-object importance scores already computed, loading from file")
        return np.load('Ranking_arrays/obj_x_obj_authenticity_importance_scores.npy', allow_pickle=True)
    
    model.eval()
    model.to(device)
    named_modules = list(model.named_modules())
    layer = None
    
    for name, module in named_modules:
        if name == layer_name:
            layer = module
            break
    
    if layer is None:
        raise ValueError(f"Layer {layer_name} not found in model")
    
    num_channels = layer.out_channels
    
    # Make sure batch_size=1 in the dataloader
    if dataloader.batch_size != 1:
        print("Warning: Dataloader batch size should be 1 for per-object importance computation")
    
    # Create a directory for saving results if it doesn't exist
    os.makedirs('Ranking_arrays', exist_ok=True)
    
    # Initialize array to store importance scores arrays for each image
    num_images = len(dataloader)
    importance_array = np.empty(num_images, dtype=object)
    
    # Process each image individually
    for img_idx, (inputs, labels) in enumerate(dataloader):
        print(f"Processing image {img_idx}/{num_images}")
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Get baseline prediction
        with torch.no_grad():
            baseline_outputs, _ = model(inputs)
            baseline_error = torch.abs(baseline_outputs - labels).item()
        
        # Initialize array for this image's channel importance scores
        channel_scores = np.zeros(num_channels)
        
        # Compute importance for each feature map for this image
        for channel_idx in range(num_channels):
            # Create a backup of the weights and bias
            backup_weights = layer.weight[channel_idx, ...].clone()
            backup_bias = layer.bias[channel_idx].clone() if layer.bias is not None else None
            
            # Zero out the channel_idx-th output channel
            with torch.no_grad():
                layer.weight[channel_idx, ...] = 0
                if layer.bias is not None:
                    layer.bias[channel_idx] = 0
                
            # Get prediction with the pruned feature map
            with torch.no_grad():
                pruned_outputs, _ = model(inputs)
                pruned_error = torch.abs(pruned_outputs - labels).item()
            
            # Compute importance score
            importance_score = pruned_error - baseline_error
            
            # Store the importance score in the array
            channel_scores[channel_idx] = importance_score
            
            # Restore weights and bias
            with torch.no_grad():
                layer.weight[channel_idx, ...] = backup_weights
                if layer.bias is not None:
                    layer.bias[channel_idx] = backup_bias
        
        # Store the channel scores array for this image
        importance_array[img_idx] = channel_scores
    
    # Save results as a numpy array
    np.save('Ranking_arrays/obj_x_obj_authenticity_importance_scores.npy', importance_array)
    
    # Return the array of arrays
    return importance_array

In [15]:

# LAYER to prune 
LAYER = 'features.28'  # Last convolutional layer (assuming it has 512 channels)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Make sure batch size is 1
BATCH_SIZE = 1


# Load the model
model = AuthenticityPredictor()
model.load_state_dict(torch.load('Weights/VGG-16_real_authenticity_finetuned.pth'))
model.eval()
model.to(DEVICE)

# Compute per-object feature importance
obj_x_obj_importance = compute_obj_x_obj_feature_map_importance(model, dataloaders['test'], DEVICE, LAYER)





/tmp/ipykernel_1774801/3334411481.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Weights/VGG-16_real_authenticity_finetuned.pth'))


Processing image 0/138
Processing image 1/138
Processing image 2/138
Processing image 3/138
Processing image 4/138
Processing image 5/138
Processing image 6/138
Processing image 7/138
Processing image 8/138
Processing image 9/138
Processing image 10/138
Processing image 11/138
Processing image 12/138
Processing image 13/138
Processing image 14/138
Processing image 15/138
Processing image 16/138
Processing image 17/138
Processing image 18/138
Processing image 19/138
Processing image 20/138
Processing image 21/138
Processing image 22/138
Processing image 23/138
Processing image 24/138
Processing image 25/138
Processing image 26/138
Processing image 27/138
Processing image 28/138
Processing image 29/138
Processing image 30/138
Processing image 31/138
Processing image 32/138
Processing image 33/138
Processing image 34/138
Processing image 35/138
Processing image 36/138
Processing image 37/138
Processing image 38/138
Processing image 39/138
Processing image 40/138
Processing image 41/138
Pr